In [239]:
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression
import imutils


In [240]:
# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cv2.startWindowThread()

# open webcam video stream
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    
    frame = imutils.resize(frame, width=min(400, frame.shape[1]))
    
    # detect people in the image
    # returns the bounding boxes for the detected objects
    boxes, weights = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05)

    boxes = np.array([[x, y, x + w, y + h] for (x, y, w, h) in boxes])
    pick = non_max_suppression(boxes, probs=None, overlapThresh=0.65)
    
    for (xA, yA, xB, yB) in pick:
        print(xA, yA, xB, yB)
        # display the detected boxes in the colour picture
        cv2.rectangle(frame, (xA, yA), (xB, yB),
                          (0, 255, 0), 2)
    
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()

# finally, close the window
cv2.destroyAllWindows()
cv2.waitKey(1)

95 78 163 214
98 76 166 212
102 78 169 212
210 106 277 240
203 99 277 245
110 0 250 273
112 22 243 283
107 22 242 292
16 138 81 268
106 24 242 295
102 15 247 300
102 15 247 300
118 48 241 293
194 0 259 125
118 41 242 289
114 35 243 294
119 44 242 289
194 0 259 124
108 27 242 295
111 32 241 292
109 30 240 292
105 27 239 294
103 20 241 294
108 33 239 294
98 12 241 297
104 24 240 295
106 27 239 293
100 19 238 295
103 23 239 294
100 14 240 294
100 14 240 294
13 132 81 269
100 13 241 294
102 17 240 293
100 14 241 295
8 122 83 273
101 14 241 292
9 124 82 270
99 5 243 292
101 5 242 286
101 4 242 286
107 8 244 281
111 0 251 275
121 0 252 260
195 0 259 125


-1

In [33]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [241]:

detection_model = tf.saved_model.load("/media/ncostamagna/Costamagna/Proyectos/Axul/axul_monitor/back/ssd_inception_v2_coco_2017_11_17/saved_model")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [57]:

category_index = label_map_util.create_category_index_from_labelmap('mscoco_label_map.pbtxt', use_display_name=True)


{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

In [237]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]
 
  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)
 
  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections
 
  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
     
  return output_dict

def show_inference(model, frame):
  #take the frame from webcam feed and convert that to array
  image_np = np.array(frame)
  # Actual detection.
     
  output_dict = run_inference_for_single_image(model, image_np)
  
  if output_dict['detection_scores'][0] > np.float64(0.5):
    arriba = output_dict['detection_boxes'][0][0]
    izquiera = output_dict['detection_boxes'][0][1]
    abajo = output_dict['detection_boxes'][0][2]
    derecha = output_dict['detection_boxes'][0][3]
    print("arriba={} abajo={} izquiera={} derecha={}".format(arriba,abajo,izquiera,derecha))
    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=5)

  return(image_np)

In [238]:
import cv2
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    re,frame = video_capture.read()
    Imagenp=show_inference(detection_model, frame)
    cv2.imshow('object detection', cv2.resize(Imagenp, (800,800)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()

arriba=0.43532097339630127 abajo=0.989403247833252 izquiera=0.13253110647201538 derecha=0.906982958316803
arriba=0.42907407879829407 abajo=0.9899647235870361 izquiera=0.1165894865989685 derecha=0.8825158476829529
arriba=0.4287656247615814 abajo=0.9892823696136475 izquiera=0.13094466924667358 derecha=0.8982217907905579
arriba=0.4327201247215271 abajo=0.9896965622901917 izquiera=0.12980729341506958 derecha=0.898912250995636
arriba=0.4296590983867645 abajo=0.9935017824172974 izquiera=0.13215360045433044 derecha=0.9002493619918823
arriba=0.44213712215423584 abajo=0.9942017793655396 izquiera=0.17795705795288086 derecha=0.8940155506134033
arriba=0.443324476480484 abajo=0.990670919418335 izquiera=0.12084832787513733 derecha=0.886418342590332
arriba=0.4456118047237396 abajo=0.992300271987915 izquiera=0.10900750756263733 derecha=0.8761184215545654
arriba=0.4294624626636505 abajo=0.9898223876953125 izquiera=0.11757513880729675 derecha=0.8833494186401367
arriba=0.4291939437389374 abajo=0.98750710

arriba=0.2312052845954895 abajo=1.0 izquiera=0.4940573573112488 derecha=0.8800144791603088
arriba=0.2359713315963745 abajo=1.0 izquiera=0.48233288526535034 derecha=0.8814881443977356
arriba=0.23135080933570862 abajo=0.9996628761291504 izquiera=0.48783761262893677 derecha=0.8645828366279602
arriba=0.24241498112678528 abajo=0.997901439666748 izquiera=0.48574814200401306 derecha=0.8757718801498413
arriba=0.24112758040428162 abajo=0.9990730285644531 izquiera=0.47655802965164185 derecha=0.8776970505714417
arriba=0.24149104952812195 abajo=0.9976999759674072 izquiera=0.4756295084953308 derecha=0.8743428587913513
arriba=0.24688410758972168 abajo=1.0 izquiera=0.48547428846359253 derecha=0.873698890209198
arriba=0.2466317117214203 abajo=1.0 izquiera=0.481154203414917 derecha=0.8700336217880249
arriba=0.25509533286094666 abajo=1.0 izquiera=0.4690646529197693 derecha=0.8616556525230408
arriba=0.24227967858314514 abajo=0.998540997505188 izquiera=0.4713141918182373 derecha=0.8518739938735962
arriba=

arriba=0.11004224419593811 abajo=0.9980114698410034 izquiera=0.02368992567062378 derecha=0.9866496920585632
arriba=0.10388508439064026 abajo=0.9929949045181274 izquiera=0.03038749098777771 derecha=0.99275803565979
arriba=0.09384971857070923 abajo=0.9963162541389465 izquiera=0.0448555052280426 derecha=0.9888646602630615
arriba=0.06770867109298706 abajo=0.995892345905304 izquiera=0.0814075767993927 derecha=0.9916901588439941
arriba=0.04726371169090271 abajo=0.9927347898483276 izquiera=0.06441792845726013 derecha=0.9949971437454224
arriba=0.0336686372756958 abajo=0.9898480176925659 izquiera=0.07095456123352051 derecha=0.995320200920105
arriba=0.006688892841339111 abajo=0.9930987358093262 izquiera=0.05283573269844055 derecha=0.9948490858078003
arriba=0.0044571757316589355 abajo=0.9951330423355103 izquiera=0.032559871673583984 derecha=0.9944612979888916
